In [1]:
from audiocraft.modules.semcodec import SemCodecMidiDecoder

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.8.18 (you have 3.8.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/jongmin/.local/share/virtualenvs/audiocraft-ssbSDm-j/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
model = SemCodecMidiDecoder()

In [4]:
tokens = torch.load("/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV_encodec.pt")

In [5]:
token = tokens[60]
token.shape

torch.Size([1, 4, 1500])

In [6]:
out = model(token)
out.shape

torch.Size([1, 2, 88, 1500])

In [7]:
from audiocraft.losses import note_eval

In [8]:
notes = note_eval.extract_notes(out[0,0].T, out[0,1].T, None)

In [9]:
pitches = notes[0]
intervals = notes[1]

In [10]:
pitches[0]

3

In [11]:
import pretty_midi

In [12]:
midi = pretty_midi.PrettyMIDI()

In [13]:
midi.instruments.append(pretty_midi.Instrument(0))

In [14]:
for i, pitch in enumerate(pitches):
    midi.instruments[0].notes.append(pretty_midi.Note(
        velocity=100,
        pitch=pitch,
        start=intervals[i,0]/50,
        end=intervals[i,1]/50
    ))
midi.instruments[0].notes

[Note(start=0.000000, end=0.460000, pitch=3, velocity=100),
 Note(start=0.000000, end=0.260000, pitch=4, velocity=100),
 Note(start=0.000000, end=0.020000, pitch=5, velocity=100),
 Note(start=0.000000, end=0.040000, pitch=13, velocity=100),
 Note(start=0.000000, end=0.140000, pitch=14, velocity=100),
 Note(start=0.000000, end=1.100000, pitch=16, velocity=100),
 Note(start=0.000000, end=0.020000, pitch=18, velocity=100),
 Note(start=0.000000, end=0.520000, pitch=19, velocity=100),
 Note(start=0.000000, end=0.100000, pitch=21, velocity=100),
 Note(start=0.000000, end=0.500000, pitch=22, velocity=100),
 Note(start=0.000000, end=0.300000, pitch=24, velocity=100),
 Note(start=0.000000, end=0.200000, pitch=25, velocity=100),
 Note(start=0.000000, end=0.140000, pitch=28, velocity=100),
 Note(start=0.000000, end=0.040000, pitch=29, velocity=100),
 Note(start=0.000000, end=0.520000, pitch=30, velocity=100),
 Note(start=0.000000, end=0.560000, pitch=31, velocity=100),
 Note(start=0.000000, end=0

In [15]:
import muspy

In [16]:
mus = muspy.from_pretty_midi(midi)

In [17]:
muspy.download_musescore_soundfont()

Skip downloading as the MuseScore General soundfont is found.


In [19]:
mus_synth = mus.synthesize()

In [21]:
mus_synth.shape

(1322688, 2)

In [22]:
import IPython.display as ipd

In [25]:
ipd.Audio(mus_synth.T[0], rate=44100)